In [62]:
import pandas as pd
df = pd.read_csv('../data/Original_csv/Hyperion.csv', na_filter=False)

In [63]:
import re
#df.drop('Domanda', inplace=True, axis=1)
#df.drop('Repertorio', inplace=True, axis=1)

#filter = df["Repertorio"] != ""
#df = df[filter]

# lower entire dataset
#df = df.applymap(str.lower)

#df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
#df['Repertorio'].replace('previsioni','previsione', inplace=True)



In [79]:
import string

dataset = []
sample = {}

for row in df.itertuples():
    if row.Testo:
        dataset.append(sample)
        sample = {}
        sample['Testo'] = row.Testo.replace("\n", " ")
        sample['Stralci'] = [row.Stralcio.replace("\n", " ")]
        sample['Repertori'] = [row.Repertorio]
        
    if not row.Testo:
        sample['Stralci'].append(row.Stralcio.replace("\n", " "))
        sample['Repertori'].append(row.Repertorio)
del dataset[0]

#Find bounds starting froma text
def find_char_bounds(spans: list, text: str) -> list:
    bounds = []
    last_char = 0
    for span in spans:
        start = text.find(span)
        if start == -1:
            start = last_char + 1
        bounds.append((start, start + len(span)))
        last_char = start + len(span)
    return bounds

def find_word_bounds(spans: list, text: str) -> list:
    bounds = []
    end = 0
    for span in spans:
        s = span.translate(str.maketrans('', '', string.punctuation))
        word_list = s.split()
        text_list = text.translate(str.maketrans('', '', string.punctuation)).split()
        try:
            start = text_list.index(word_list[0], end)
        except:
            if not bounds:
                start = 0
            else:
                
                start = bounds[-1][1] + 1
        end = start + len(word_list) - 1
            
        bounds.append((start, end))
    return bounds


for sample in dataset:
    #sample['Bounds'] = find_char_bounds(sample['Stralci'], sample['Testo'])
    sample['Bounds'] = find_word_bounds(sample['Stralci'], sample['Testo'])



In [91]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import re

nltk_pred = []
spans_pred = []

for sample in dataset:
    tokens = sent_tokenize(sample['Testo'])
    spans = []
    bounds = []
    for x in tokens:
        #spans += re.findall('.*?[.:!?;,]', x)
        spans += re.split('[.:!?;,]', x)
        spans = list(filter(None, spans)) # filter empty strings

    #bounds += find_char_bounds(spans, sample['Testo'])
    bounds += find_word_bounds(spans, sample['Testo'])
    nltk_pred.append(bounds)
    spans_pred.append(spans) 

[nltk_data] Downloading package punkt to /home/michele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [92]:
print(dataset[1100]['Testo'])
print(dataset[1100]['Bounds'])
print(nltk_pred[1100])
print(spans_pred[1100])

Zaia, 0,4% positivi su 1200 lavoratori In settimana screening su altri 13 mila di 79 aziende VENEZIA, 10 MAG - Cinque lavoratori su 1.200, di otto aziende padovane, sono risultati positivi al Coronavirus. Lo 0,4% ha sottolineato il presidente del Veneto, Luca Zaia, il quale ha annunciato che in settimana partirà uno screening in altre 79 aziende regionali per complessivi 13.000 dipendenti.     "E' un progetto pilota - ha ricordato l'assessore alla sanità del Veneto, Manuela Lanzarin - al quale avevano aderito 8 aziende del padovano e ora la richiesta si è allargata alle altre". L'assessore ha infine rilevato che "si stanno concludendo i test svolti a farmacisti e loro collaboratori, personale delle forze dell' ordine, volontari della protezione Civile, sanitari. Potremo così avere un dato preciso per fasce di ogni singola categoria".
[(0, 39), (40, 60), (61, 116), (117, 128)]
[(0, 0), (1, 1), (2, 17), (17, 22), (23, 23), (23, 26), (27, 31), (32, 33), (34, 40), (40, 41), (42, 59), (60, 

In [93]:
import numpy as np
# A è B sono tupe con i bound dello span
def IoU(A, B):
    if A == B:
        return 1
    start = max(A[0], B[0])
    end = min(A[1], B[1])
    if(start > end):
        return 0
    intersection = end - start
    return intersection / (A[1] - A[0] + B[1] - B[0] - intersection)

def compute_IoUs(pred_bounds, gt_spans):
    IoUs = []
    for gt_bounds in gt_spans:
        IoUs.append(IoU(pred_bounds, gt_bounds)) 
    return IoUs

#Input: text_spans_dict = [ {
#           'Bounds' : (a,b), 
#           'IoU' : float,
#           'Repertorio': 'string':
#           } ]
def normalize(text_spans_dict, gt_spans):
    normalized = []
    for i in range(len(text_spans_dict)):
        #normalized is not empty
        if normalized:
            if normalized[-1]['Repertorio'] == text_spans_dict[i]['Repertorio']:
                new_span = (normalized[-1]['Bounds'][0], text_spans_dict[i]['Bounds'][1])
                new_span_features = {
                    'Bounds' : new_span, 
                    'IoU' : None,
                    'Repertorio' : text_spans_dict[i]['Repertorio']
                    }
                del normalized[-1]
                normalized.append(new_span_features)
            else:
                normalized.append(text_spans_dict[i])
        else:
            normalized.append(text_spans_dict[i])
        
    
    for i in range(len(normalized)):
        normalized[i]['IoU'] = max(compute_IoUs(normalized[i]['Bounds'], gt_spans['Bounds']))
    return normalized
    

metrics = []
normalized_metrics = []
for i, pred_bounds in enumerate(nltk_pred):
    text_IoUs = []
    for pred_span in pred_bounds:
        IoUs = compute_IoUs(pred_span, dataset[i]['Bounds'])
        best = np.argmax(IoUs)
        span_features = {
            'Bounds' : pred_span, 
            'IoU' : IoUs[best],
            'Repertorio' : dataset[i]['Repertori'][best]
            }

        text_IoUs.append(span_features)
    metrics.append(text_IoUs)
    normalized_metrics.append(normalize(text_IoUs, dataset[i]))


        
#TODO non funziona normalizzazione


In [94]:
print(metrics[10])
print(normalized_metrics[10])
print(dataset[10])

[{'Bounds': (0, 7), 'IoU': 0.7, 'Repertorio': 'Non risposta'}, {'Bounds': (8, 10), 'IoU': 0.2, 'Repertorio': 'Non risposta'}, {'Bounds': (11, 13), 'IoU': 0.6666666666666666, 'Repertorio': 'Commento'}, {'Bounds': (14, 14), 'IoU': 0, 'Repertorio': 'Non risposta'}, {'Bounds': (15, 22), 'IoU': 1, 'Repertorio': 'Giudizio'}]
[{'Bounds': (0, 10), 'IoU': 1, 'Repertorio': 'Non risposta'}, {'Bounds': (11, 13), 'IoU': 0.6666666666666666, 'Repertorio': 'Commento'}, {'Bounds': (14, 14), 'IoU': 0, 'Repertorio': 'Non risposta'}, {'Bounds': (15, 22), 'IoU': 1, 'Repertorio': 'Giudizio'}]
{'Testo': 'Chi non vuole il MES scenda in piazza. Coronavirus o no. Basta social network. Basta. Siamo uomini e donne o larve da tastiera?', 'Stralci': ['Chi non vuole il MES scenda in piazza. Coronavirus o no. ', 'Basta social network. Basta', 'Siamo uomini e donne o larve da tastiera?'], 'Repertori': ['Non risposta', 'Commento', 'Giudizio'], 'Bounds': [(0, 10), (11, 14), (15, 22)]}


In [95]:
n_spans = 0
for e in dataset:
    n_spans += len(e['Bounds'])
print('Numero stralci nel dataset:', str(n_spans))

n_spans = 0
for e in normalized_metrics:
    n_spans += len(e)
print('Numero stralci predetti:', str(n_spans))

mean = 0
long_spans = 0
min_lenght = 0
perfect_spans =0
for text in normalized_metrics:
    for span in text:
        if span['Bounds'][1] - span['Bounds'][0] >= min_lenght:
            long_spans += 1
            mean += span['IoU']
            if span['IoU'] == 1:
                perfect_spans += 1
perfect_spans_perc = perfect_spans / long_spans
mean_IoU = mean / long_spans
print('Numero stralci con lunghezza minima = ', str(min_lenght), ': ', str(long_spans))
print('Media IoU:', str(mean_IoU))
print('Percentuale span perfetti: ', str(perfect_spans_perc))




Numero stralci nel dataset: 35471
Numero stralci predetti: 34244
Numero stralci con lunghezza minima =  0 :  34190
Media IoU: 0.8214227711044331
Percentuale span perfetti:  0.6230184264404797


## Span Classification

I provide in input non normalized spans, to predict the label, then I apply normalization to spans exploiting predicted labels 

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained('MiBo/RepML')
tokenizer = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")


In [132]:
print(len(spans_pred[0]))
print(len(metrics[0]))
type(metrics[0][0]['Repertorio'])

6
6


str

In [133]:


spans = []
labels = []
for span in spans_pred:
    spans += span
for text in metrics:
    for span in text:
        labels.append(span['Repertorio'])
print(len(spans))
print(len(labels))

df = pd.DataFrame.from_records({
    'Stralcio' : spans,
    'Repertorio' : labels
})



151230
151230


In [ ]:
import re

import pandas as pd
import torch
from sklearn import preprocessing
from transformers import AutoTokenizer

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

LABELS = [
                'anticipazione',
                'causa',
                'commento',
                'conferma',
                'considerazione',
                'contrapposizione',
                'deresponsabilizzazione',
                'descrizione',
                'dichiarazione di intenti',
                'generalizzazione',
                'giudizio',
                'giustificazione',
                'implicazione',
                'non risposta',
                'opinione',
                'possibilità',
                'prescrizione',
                'previsione',
                'proposta',
                'ridimensionamento',
                'sancire',
                'specificazione',
                'valutazione'
        ]

class HyperionDataset(torch.utils.data.Dataset):
    

    def __init__(self, df, tokenizer_name):
        #fill_null_features(df)
        df = filter_empty_labels(df)
        #df = twitter_preprocess(df)
        df = to_lower_case(df)
        uniform_labels(df)          
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.encodings = tokenize(df, tokenizer)
        self.labels = encode_labels(df).tolist()    

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
    
    def labels_list(self):
        return LABELS



# Dataset loading and preprocessing
def fill_null_features(df):
        for c in ['Domanda','Testo']:
            for i in range(0,len(df.index)):  
                if not df[c][i]:
                    j=i
                    while j>0: 
                        j-=1
                        if df[c][j]:
                            df[c][i] = df[c][j]
                            break

#Delete examples with empty label
def filter_empty_labels(df):
    filter = df["Repertorio"] != ""
    return df[filter]

#Convert to lower case
def to_lower_case(df):
    return df.applymap(str.lower)


#Lables uniformation uncased
def uniform_labels(df):
    df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
    df['Repertorio'].replace('previsioni','previsione', inplace=True)

def tokenize(df, tokenizer):
    return tokenizer(
        df['Stralcio'].tolist(),
        #df['Domanda'].tolist(),
        max_length=512,
        add_special_tokens=True,
        return_attention_mask=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

def encode_labels(df):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.transform(df['Repertorio'])

def decode_labels(encoded_labels):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.inverse_transform(encoded_labels)

def twitter_preprocess(df):
    text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
    )

    processed = []

    for s in df['Stralcio']:
        s = str(" ".join(text_processor.pre_process_doc(s)))
        s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
        s = re.sub(r"\s+", ' ', s)
        s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
        s = re.sub ( r'^\s' , '' , s )
        s = re.sub ( r'\s$' , '' , s )
        processed.append(s)

    df['Stralcio'] = processed

    return df
    

def train_val_split(df, tok_name,  val_perc=0.2, subsample = False):
    gb = df.groupby('Repertorio')
    train_list = []
    val_list = []
    for x in gb.groups:
        if subsample:
            class_df = gb.get_group(x).head(50)
        else:
            class_df = gb.get_group(x)

        # Validation set creation
        val = class_df.sample(frac=val_perc)
        train = pd.concat([class_df,val]).drop_duplicates(keep=False)

        #train_list.append(train.head(500))
        train_list.append(train)
        val_list.append(val)


    train_df = pd.concat(train_list)
    val_df = pd.concat(val_list)
    return HyperionDataset(train_df, tok_name), HyperionDataset(val_df, tok_name)

In [134]:
dataset = 

,Repertorio,Stralcio
0,Dichiarazione di intenti,Dunque vediamo se ho capito
1,Sancire,Conte le canta a Salvini e alla Meloni
2,Giustificazione,questi reagiscono e le cantano a loro volta a...
3,Sancire,Mentana si smarca da Conte e finisce con foto...
4,Commento,Siamo tornati per un attimo alla normalità


In [135]:
import re

import pandas as pd
import torch
from sklearn import preprocessing
from transformers import AutoTokenizer

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

LABELS = [
                'anticipazione',
                'causa',
                'commento',
                'conferma',
                'considerazione',
                'contrapposizione',
                'deresponsabilizzazione',
                'descrizione',
                'dichiarazione di intenti',
                'generalizzazione',
                'giudizio',
                'giustificazione',
                'implicazione',
                'non risposta',
                'opinione',
                'possibilità',
                'prescrizione',
                'previsione',
                'proposta',
                'ridimensionamento',
                'sancire',
                'specificazione',
                'valutazione'
        ]

class HyperionDataset(torch.utils.data.Dataset):
    

    def __init__(self, df, tokenizer_name):
        #fill_null_features(df)
        df = filter_empty_labels(df)
        #df = twitter_preprocess(df)
        df = to_lower_case(df)
        uniform_labels(df)          
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.encodings = tokenize(df, tokenizer)
        self.labels = encode_labels(df).tolist()    

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
    
    def labels_list(self):
        return LABELS



# Dataset loading and preprocessing
def fill_null_features(df):
        for c in ['Domanda','Testo']:
            for i in range(0,len(df.index)):  
                if not df[c][i]:
                    j=i
                    while j>0: 
                        j-=1
                        if df[c][j]:
                            df[c][i] = df[c][j]
                            break

#Delete examples with empty label
def filter_empty_labels(df):
    filter = df["Repertorio"] != ""
    return df[filter]

#Convert to lower case
def to_lower_case(df):
    return df.applymap(str.lower)


#Lables uniformation uncased
def uniform_labels(df):
    df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
    df['Repertorio'].replace('previsioni','previsione', inplace=True)

def tokenize(df, tokenizer):
    return tokenizer(
        df['Stralcio'].tolist(),
        #df['Domanda'].tolist(),
        max_length=512,
        add_special_tokens=True,
        return_attention_mask=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

def encode_labels(df):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.transform(df['Repertorio'])

def decode_labels(encoded_labels):
    le = preprocessing.LabelEncoder()
    le.fit(LABELS)
    return le.inverse_transform(encoded_labels)

def twitter_preprocess(df):
    text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
    )

    processed = []

    for s in df['Stralcio']:
        s = str(" ".join(text_processor.pre_process_doc(s)))
        s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
        s = re.sub(r"\s+", ' ', s)
        s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
        s = re.sub ( r'^\s' , '' , s )
        s = re.sub ( r'\s$' , '' , s )
        processed.append(s)

    df['Stralcio'] = processed

    return df
    

def train_val_split(df, tok_name,  val_perc=0.2, subsample = False):
    gb = df.groupby('Repertorio')
    train_list = []
    val_list = []
    for x in gb.groups:
        if subsample:
            class_df = gb.get_group(x).head(50)
        else:
            class_df = gb.get_group(x)

        # Validation set creation
        val = class_df.sample(frac=val_perc)
        train = pd.concat([class_df,val]).drop_duplicates(keep=False)

        #train_list.append(train.head(500))
        train_list.append(train)
        val_list.append(val)


    train_df = pd.concat(train_list)
    val_df = pd.concat(val_list)
    return HyperionDataset(train_df, tok_name), HyperionDataset(val_df, tok_name)